In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from matplotlib import pyplot as plt
import numpy as np
import re

<p style="border:3px #ff0000  solid;">
1. Загрузите данные из csv файла. Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков.

In [3]:
grant_df = pd.read_csv('C:/Users/Z/PycharmProjects/data_science_1t/task_2.4/grant_data_imb.csv')
#Крутое исправление заголовков
grant_df.columns = [re.sub('\\.+', '_', i).lower() for i in grant_df.columns]
#В датафрейме есть дублирующиеся колонки A..1 и A.1, значения которых отличаются, поэтому сохраним обе
grant_df = grant_df.rename(columns={"A..1": "a_1", "A.1": "b_1", "B.1": "c_1", "C.1": "d_1",})

In [4]:
grant_df

,grant_status,sponsor_code,grant_category_code,contract_value_band_see_note_a,rfcd_code_1,rfcd_percentage_1,rfcd_code_2,rfcd_percentage_2,rfcd_code_3,rfcd_percentage_3,...,dept_no_1,faculty_no_1,with_phd_1,no_of_years_in_uni_at_time_of_grant_1,number_of_successful_grant_1,number_of_unsuccessful_grant_1,a_1,a_1,b_1,c_1
0,0,97A,30B,A,321024.0,50.0,321013.0,30.0,291502.0,20.0,...,2563.0,25.0,Yes,>10 to 15,2.0,6.0,3.0,5.0,15.0,3.0
1,0,36D,10A,G,300201.0,100.0,0.0,0.0,0.0,0.0,...,1038.0,1.0,NaN,Less than 0,0.0,3.0,0.0,4.0,0.0,0.0
2,0,317A,30D,NaN,321013.0,100.0,0.0,0.0,0.0,0.0,...,2763.0,25.0,Yes,>5 to 10,4.0,3.0,6.0,25.0,14.0,14.0
3,0,62B,10B,B,321103.0,30.0,321105.0,40.0,321204.0,30.0,...,2848.0,25.0,NaN,Less than 0,1.0,2.0,1.0,0.0,0.0,0.0
4,0,1A,10A,NaN,270603.0,60.0,321205.0,30.0,320603.0,10.0,...,2678.0,25.0,Yes,>5 to 10,5.0,14.0,0.0,9.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4108,0,103C,30B,NaN,321022.0,100.0,0.0,0.0,0.0,0.0,...,2773.0,25.0,NaN,>=0 to 5,0.0,1.0,0.0,0.0,0.0,0.0
4109,0,2B,10A,NaN,340208.0,50.0,340499.0,50.0,0.0,0.0,...,1678.0,13.0,Yes,more than 15,0.0,1.0,1.0,5.0,8.0,0.0
4110,1,2B,10A,NaN,320702.0,40.0,270102.0,30.0,320305.0,30.0,...,2653.0,25.0,NaN,>=0 to 5,1.0,0.0,26.0,15.0,9.0,6.0
4111,0,40D,10B,C,NaN,NaN,NaN,NaN,NaN,NaN,...,3248.0,31.0,NaN,>=0 to 5,0.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
grant_df.info()

In [ ]:
grant_df.describe().T

In [ ]:
grant_df.isna().sum()

<p style="border:3px #ff0000  solid;">
2. Подготовьте данные к обучению моделей:

     b) Заполните пропуски
          в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта),
          в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;

In [ ]:
#заполним country_of_birth_1 самым частовстречающимся значением
val_country = grant_df.country_of_birth_1.mode()[0]
grant_df['country_of_birth_1'] = grant_df['country_of_birth_1'].fillna(val_country)

In [ ]:
#заполним данные home_language_1 на основе данных country_of_birth_1
print(grant_df['home_language_1'].isna().sum())
def languages_of_country(country):
    if country == 'Asia Pacific' or country == 'Eastern Europe' or country == 'Middle East and Africa' or country == 'South Africa':
        return 'Other'
    else:
        return 'English'
for i, row in grant_df.iterrows():
    if pd.isna(row['home_language_1']):
        grant_df.at[i, 'home_language_1'] = languages_of_country(row['country_of_birth_1'])
print(grant_df['home_language_1'].isna().sum())

In [ ]:
#Заполним остальные пропуски
grant_df.sponsor_code.fillna('N/A', inplace=True)
grant_df.grant_category_code.fillna('N/A', inplace=True)
grant_df.contract_value_band_see_note_a.fillna('N', inplace=True)
grant_df.with_phd_1.fillna('No', inplace=True)
grant_df.no_of_years_in_uni_at_time_of_grant_1.fillna('N/A', inplace=True)
grant_df.dept_no_1.fillna(grant_df.dept_no_1.mode()[0], inplace=True)
grant_df.faculty_no_1.fillna(grant_df.faculty_no_1.mode()[0], inplace=True)
grant_df.role_1.fillna('N/A', inplace=True)

In [ ]:
grant_df.fillna(0, inplace=True)

In [ ]:
grant_df.info()

<p style="border:3px #00B344  solid;">
Пропусков не осталось
</p>

     a) Отделите целевую переменную Grant.Status и выясните, сбалансированы ли классы. 
     Если классы не сбалансированы, используйте в работе хотя бы один из изученных методов борьбы с дисбалансом классов

In [ ]:
target_grant_status = grant_df['grant_status']
features = grant_df.drop(['grant_status'], axis=1)
#Смотрим распределение классов по целевому признаку
target_grant_status.value_counts()

<p style="border:3px #00B344  solid;">
<b>Вывод:</b> Распределение классов целевого признака не сбалансировано. В дальнейшей работе будем применять методы борьбы с дисбалансом.
</p>

     c) Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;

In [ ]:
features_df_without_category = pd.get_dummies(features, drop_first=True)
features_df_without_category.head()

     d) Разделите данные на обучающую и тестовую части;

In [ ]:
#Разделяем датафрейм на тренировочную и валидационную части
features_train, features_test, target_grant_train, target_grant_test = train_test_split(features_df_without_category, 
                                                                                        target_grant_status, 
                                                                                        test_size=0.25, 
                                                                                        random_state=25)

     e) Используйте масштабирование для получения признаков одинакового масштаба

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_sc = scaler.transform(features_train)
features_valid_sc = scaler.transform(features_test)

<p style="border:3px #ff0000  solid;">
3. Обучите модели и выберите лучшую:

     a)Обучите модель логистической регрессии, используя кросс-валидацию. Оцените ее качество с помощью метрики rocauc. Выведите топ-10 признаков по важности, согласно обученной модели;

In [ ]:
#Так как распределение классов целевого признака не сбалансировано, при инициализации модели обязательно укажем параметр class_weight='balanced'
model_Log_Reg_CV = LogisticRegressionCV(solver='liblinear', 
                                   random_state=12, 
                                   class_weight='balanced',
                                   cv=10)
model_Log_Reg_CV.fit(features_train_sc, target_grant_train)
predict_Log_Reg_CV_proba = model_Log_Reg_CV.predict_proba(features_valid_sc)[:, 1]
roc_auc_score(target_grant_test, predict_Log_Reg_CV_proba)


In [ ]:
fpr, tpr, thresholds = roc_curve(target_grant_test, predict_Log_Reg_CV_proba)
plt.figure(figsize=(7, 7))
plt.step(fpr, tpr, where='post')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Кривая ROC')
plt.show()

In [ ]:
#ТОП-10 признаков по важности, согласно обученной модели
pd.DataFrame(zip(list(features.columns), list(abs(model_Log_Reg_CV.coef_[0]))), 
             columns=['Признак', 'Важность']).sort_values(by=['Важность'], 
                                                          ascending=False).head(10)

     b) Обучите модель случайного леса
          Для подбора гиперпараметров и кросс-валидации используйте структуру GridSearchCV,
          Выберите наилучший вариант случайного леса и выведите его параметры,
          Оцените качество выбранной модели с помощью метрики rocauc,
          Выведите топ-10 признаков по важности. Используйте атрибут feature_importances_, чтобы узнать важность признаков в деревянных моделях.

In [ ]:
#GridSearchCV
hyp_param = {'n_estimators' : range(2, 11, 2),
             'max_depth' : range(1, 11, 2),
             'min_samples_leaf' : range(1, 8),
             'min_samples_split' : range(2, 10, 2)
             }

In [ ]:
model_RFC = RandomForestClassifier(random_state=25)
grid_search = GridSearchCV(model_RFC, hyp_param, cv=5)
grid_search.fit(features_train_sc, target_grant_train)
grid_search.best_params_

In [ ]:
predict_model_RFC_proba = grid_search.predict_proba(features_valid_sc)
roc_auc_score(target_grant_test, predict_model_RFC_proba[:, 1])

In [ ]:
fpr, tpr, thresholds = roc_curve(target_grant_test, predict_model_RFC_proba[:, 1])
plt.figure(figsize=(7, 7))
plt.step(fpr, tpr, where='post')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Кривая ROC')
plt.show()

In [ ]:
pd.DataFrame(zip(list(features.columns), list(grid_search.best_estimator_.feature_importances_)), 
             columns=['Признак', 'Важность']).sort_values(by=['Важность'], 
                                                          ascending=False).head(10)

<p style="border:3px #ff0000  solid;">
4. Напишите вывод, в котором будет содержаться информация о том, какие признаки важны согласно обеим моделям, и какая модель оказалась наилучшей для решения поставленной задачи.

<p style="border:3px #00B344  solid;">
<b>Выводы: </b> 
    
    1. Сравнивая значения метрик rocauc, можно сделать вывод что наилучшее качество имеет модель логистической регрессии (0.88 против 0.86 для случайного леса).
    2. Для обеих моделей самыми важными признаками выделены year_of_birth_1 и role_1. 
    3. Признак sponsor_code так же попал в топ-10 важных признаков для обеих моделей.
    4. Очень важным этапом, влияющим на конечный результат - является подготовка данных, в том числе правильное заполнение пропусков. 
    5. Заполнению пропусков стоит уделять внимания не меньше, чем подбору гиперпараметров.
</p>